In [1]:
!pip install ultralytics==8.0.196
!pip install roboflow

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
from google.colab import drive
drive.mount('/content/drive')
ROOT_DIR = '/content/drive/MyDrive/Stamp_remover'
%cd {ROOT_DIR}

Mounted at /content/drive
/content/drive/MyDrive/Stamp_remover


In [6]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/Stamp_remover/runs/detect/train/weights/best.pt conf=0.4 source=/content/drive/MyDrive/frotated_image save_txt=True agnostic_nms=True  save_conf=True line_width=1

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 268 layers, 68124531 parameters, 0 gradients, 257.4 GFLOPs

image 1/111 /content/drive/MyDrive/frotated_image/Scan_20240729 (2)_page_1.jpg: 352x640 1 stamp-object, 3790.0ms
image 2/111 /content/drive/MyDrive/frotated_image/Scan_20240729 (2)_page_10.jpg: 352x640 2 stamp-objects, 2837.1ms
image 3/111 /content/drive/MyDrive/frotated_image/Scan_20240729 (2)_page_11.jpg: 352x640 2 stamp-objects, 2291.9ms
image 4/111 /content/drive/MyDrive/frotated_image/Scan_20240729 (2)_page_12.jpg: 352x640 2 stamp-objects, 2351.9ms
image 5/111 /content/drive/MyDrive/frotated_image/Scan_20240729 (2)_page_13.jpg: 352x640 2 stamp-objects, 2664.9ms
image 6/111 /content/drive/MyDrive/frotated_image/Scan_20240729 (2)_page_2.jpg: 352x640 1 stamp-object, 3630.1ms
image 7/111 /content/drive/MyDrive/frotated_image/Scan_20240729 (2)_page_3.jpg: 352x640 2 stamp-objects, 2337.3ms
image 8/111 /content/drive/MyDri

In [7]:

import os
import cv2
import numpy as np

# Function to perform inpainting
def inpaint_box(image, x_min, y_min, x_max, y_max):
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    mask[y_min:y_max, x_min:x_max] = 255
    inpainted_image = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)
    return inpainted_image


# Input and output directories
input_dir = '/content/drive/MyDrive/frotated_image'
labels_dir = '/content/drive/MyDrive/Stamp_remover/runs/detect/predict4/labels'
output_dir = '/content/drive/MyDrive/Stamp_removed_data'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Loop through all images in the input directory
for image_name in os.listdir(input_dir):
    if image_name.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        image_path = os.path.join(input_dir, image_name)
        label_path = os.path.join(labels_dir, os.path.splitext(image_name)[0] + '.txt')

        # Read the image
        image = cv2.imread(image_path)

        # Read the annotation file
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                annotations = f.readlines()

            for annotation in annotations:
                parts = annotation.strip().split()
                class_id = int(parts[0])
                x_center_rel = float(parts[1])
                y_center_rel = float(parts[2])
                width_rel = float(parts[3])
                height_rel = float(parts[4])

                # Convert relative coordinates to absolute
                img_height, img_width = image.shape[:2]
                x_center = int(x_center_rel * img_width)
                y_center = int(y_center_rel * img_height)
                width = int(width_rel * img_width)
                height = int(height_rel * img_height)

                x_min = int(x_center - width / 2)
                y_min = int(y_center - height / 2)
                x_max = int(x_center + width / 2)
                y_max = int(y_center + height / 2)

                # Perform inpainting
                image = inpaint_box(image, x_min, y_min, x_max, y_max)

        # Save the inpainted image
        output_path = os.path.join(output_dir, image_name)
        cv2.imwrite(output_path, image)

print("Processing complete.")


Processing complete.
